In [ ]:
#!pip install dash 
#!pip install dash-html-components
#!pip install dash-core-components
#!pip install dash-table

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import math
import pylab 
from sklearn.linear_model import  LassoLarsIC
import statsmodels.api as sm
import scipy
import plotly.express as px

In [2]:
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import  LassoLarsIC

In [ ]:
#!pip install dash-bootstrap-components

In [194]:
#import dash_bootstrap_components as dbc 


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
#app = dash.Dash()
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
colors = {'background': '#111111','text': '#7FDBFF'}
app.layout = html.Div(children=[
    html.H1("Predicting Crop yields"),
    html.Div([dcc.Dropdown(id="dropdown_select_country",options=[
    {"label": "Netherlands", "value": "Netherlands"},
    {"label": "France", "value": "France"},
    {"label": "Germany", "value": "Germany"},
    {"label": "Switzerland", "value": "Switzerland"},
    {"label": "Austria", "value": "Austria"},
    {"label": "Poland", "value": "Poland"},
    {"label": "Italy", "value": "Italy"}],clearable=False,searchable=False,value="Germany")]),

        #html.Div([dcc.Graph(id="plot_county")], className="two-thirds column"),
        html.Div([dcc.Graph(id="plot_prediction")]),
    
        html.Div([html.H6("Select Change Maximum Temp"),
                 dcc.Slider(id="change_max_temp",
                            min=-3,
                            max=3,
                            step=None,
                            marks={-3: '-3 °C', -2: '-2 °C', -1: '-1 °C', 0: '0 °C',
                                    3: '3 °C', 2: '2 °C', 1: '1 °C'},
                            value=0),
                  html.H6("Select Change Minimum Temp"),
                 dcc.Slider(id="change_min_temp",
                            min=-3,
                            max=3,
                            step=None,
                            marks={-3: '-3 °C', -2: '-2 °C', -1: '-1 °C', 0: '0 °C',
                                    3: '3 °C', 2: '2 °C', 1: '1 °C'},
                            value=0),
                  html.H6("Select Change Amount of precip"),
                 dcc.Slider(id="change_precip",
                            min=-200,
                            max=200,
                            step=None,
                            marks={-200: '-200 mm', -150: '- 150 mm', -100: '-100 mm', 0: '0 mm',
                                    200: '200 mm', 150: '150 mm', 100: '100 mm', 0: '0 mm'},
                            value=0),
                  html.H6("Select Change Mean Presure"),
                 dcc.Slider(id="change_press",
                            min=-3,
                            max=3,
                            step=None,
                            marks={-3: '-3 ', -2: '-2 ', -1: '-1', 0: '0 ',
                                    3: '3', 2: '2', 1: '1'},
                            value=0)]),
    
    
            
                  

            html.Div([html.Div([dcc.Graph(id="plot_temp")])]),
            html.Div([html.Div([dcc.Graph(id="plot_presure")])],className="six columns"),
            html.Div([html.Div([dcc.Graph(id="plot_precip_amount")],className="six columns")])])
        
            
            
                    


'''@app.callback(
    Output("plot_county", "figure"),
    [Input("dropdown_select_country", "value")])
def update_plot_county(figure):
    data = pd.read_csv('./data/{}.csv'.format(figure), sep = ',')
    data.dropna(axis=1, inplace=True)
    #data.set_index('Year', inplace=True)
    data.drop(['Entity','Greece_pressure'], inplace=True, axis=1)
    return px.line(data, x='Year', y='Wheat (tonnes per hectare)', title="Wheat yield in {}".format(figure))'''

@app.callback(
    Output("plot_temp", "figure"),
    [Input("dropdown_select_country", "value")])
def update_plot_temp(figure):
    data = pd.read_csv('./data/{}.csv'.format(figure), sep = ',')
    data.dropna(axis=1, inplace=True)
    #data.set_index('Year', inplace=True)
    data.drop(['Entity','Greece_pressure'], inplace=True, axis=1)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Year'], y=data['Temperature_max'],
                    mode='lines+markers',
                    name='Maximum Temperature'))
    fig.add_trace(go.Scatter(x=data['Year'], y=data['Temperature_min'],
                    mode='lines+markers',
                    name='Minimum Temperature'))
    fig.update_layout(title="Min and Max Temperature in period")

    return fig

@app.callback(
    Output("plot_presure", "figure"),
    [Input("dropdown_select_country", "value")])
def update_plot_temp(figure):
    data = pd.read_csv('./data/{}.csv'.format(figure), sep = ',')
    data.dropna(axis=1, inplace=True)
    #data.set_index('Year', inplace=True)
    data.drop(['Entity','Greece_pressure'], inplace=True, axis=1)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Year'], y=data['Pressure_mean'],
                    mode='lines+markers',
                    name='Mean Presure in period'))
    fig.update_layout(title="Mean Presure in period")

    return fig
@app.callback(
    Output("plot_precip_amount", "figure"),
    [Input("dropdown_select_country", "value")])
def update_plot_temp(figure):
    data = pd.read_csv('./data/{}.csv'.format(figure), sep = ',')
    data.dropna(axis=1, inplace=True)
    #data.set_index('Year', inplace=True)
    data.drop(['Entity','Greece_pressure'], inplace=True, axis=1)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Year'], y=data['Precip_amount'],
                    mode='lines+markers',
                    name='Precip Amount'))
    fig.update_layout(title="Precip Amount in period")

    return fig


@app.callback(
    Output("plot_prediction", "figure"),
    [Input("dropdown_select_country", "value"),
    Input("change_max_temp", "value"),
     Input("change_min_temp", "value"),
     Input("change_precip", "value"),
     Input("change_press", "value")])
def update_plot_prediction(country_val,max_temp,min_temp,precip,pressure):
    data = pd.read_csv('./data/{}.csv'.format(country_val), sep = ',')
    data.dropna(axis=1, inplace=True)
    data.set_index('Year', inplace=True)
    data.drop(['Entity','Greece_pressure'], inplace=True, axis=1)
    data_train = data.loc[1961:1993]
    data_test = data.loc[1994:2018]

# Предикторы
    scaler = StandardScaler()
    #X_train = scaler.fit_transform(data_train[['Precip_amount' , 'Temperature_SAT', 'Pressure_mean', 'Temperature_min']])
    #X_test = scaler.fit_transform(data_test[['Precip_amount' , 'Temperature_SAT', 'Pressure_mean', 'Temperature_min']])
    X_train = data_train[['Precip_amount' , 'Temperature_SAT', 'Pressure_mean', 'Temperature_min']]
    X_test = data_test[['Precip_amount' , 'Temperature_SAT', 'Pressure_mean', 'Temperature_min']]
    
    y_train = data_train['Wheat (tonnes per hectare)']
    y_test = data_test['Wheat (tonnes per hectare)']

    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)
    '''LR = LinearRegression()
    LR.fit(X_train, y_train)
    predicted = LR.predict(X_test)+2'''
    
    '''model_aic = LassoLarsIC(criterion='aic')
    model_aic.fit(X_train, y_train)
    predicted_aic = model_aic.predict(X_test)+2
    predicted_aic
    '''
    clf = linear_model.Lasso(alpha=0.1)
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test+[min_temp,max_temp,pressure,precip])
    
    
       

    fig = go.Figure()
    '''fig.add_trace(go.Scatter(x=data_test.index, y=y_test,
                mode='lines+markers',
                name='Actual'))'''
    fig.add_trace(go.Scatter(x=data_test.index, y=predicted+2,
                mode='markers',
                name='Predicted'))
    fig.add_trace(go.Scatter(x=data.index, y=data['Wheat (tonnes per hectare)'],
                mode='lines+markers',
                name='Actual'))
    fig.update_layout(title="Predicted vs Actual values")
    return fig



app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Debugger PIN: 105-440-933
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
